In [18]:
import os
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import xgboost
from xgboost import XGBClassifier
from xgboostextension import XGBRanker
%matplotlib notebook

In [19]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)
pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
data_path = '/home/mvkrein/etf_model/data'
etf_data_file = os.path.join(data_path,'etf_new_var_20180910.csv')
etf_data = pd.read_csv(etf_data_file,index_col=0)

In [21]:
etf_study_file = os.path.join(data_path,'ETF_list_min_6yr_history.csv')
etf_list = pd.read_csv(etf_study_file,index_col=0)
etf_data.sort_values(['Date','sym'],ascending=True,inplace=True)
etf_data.reset_index(drop=True,inplace=True)

In [22]:
drop_columns = ['Date','sym','p', 'v', 'p_L05', 'v_L05', 'p_L10', 'v_L10', 'p_L21', 'v_L21', 'p_L42', 'v_L42', 'p_L63', \
                'v_L63', 'p_L84', 'v_L84', 'p_L126', 'v_L126', 'p_L189', 'v_L189', 'p_L252', 'v_L252',\
                'p_L-21', 'v_L-21', 'delta_p_L05', 'delta_p_L10', 'delta_p_L21', 'delta_p_L42', 'delta_p_L63', \
                'delta_p_L84', 'delta_p_L126', 'delta_p_L189', 'delta_p_L252', 'delta_p_L-21', 'delta_v_L05',\
                'delta_v_L10', 'delta_v_L21', 'delta_v_L42', 'delta_v_L63', 'delta_v_L84', 'delta_v_L126', \
                'delta_v_L189', 'delta_v_L252','rank_p_L-21', 'ivv_delta_p_L-21', 'target']

rank_col = ['rank_p_L05','rank_p_L10','rank_p_L21','rank_p_L42',\
            'rank_p_L63','rank_p_L84','rank_p_L126','rank_p_L189','rank_p_L252']

In [23]:
all_columns = list(etf_data.columns)

In [24]:
include_columns = [x for x in all_columns if x not in drop_columns]
# include_columns = ['rank_p_L05','rank_p_L10','rank_p_L21','rank_p_L42',\
#             'rank_p_L63','rank_p_L84','rank_p_L126','rank_p_L189','rank_p_L252']
# include_columns = ['rank_p_L05','rank_p_L21','rank_p_L42','rank_p_L252']

In [25]:
dates = list(etf_data['Date'].unique())
# dates[1671]

In [26]:
for i in range(0,len(dates)):
    if dates[i]=='2017-01-18':
        print(i,dates[i])

1268 2017-01-18


In [9]:
#Each year has 252 trading dates.  Need two years to fully develop variables.
dt1 = 504 #This is the first day that all variables are developed
# make all dates relative to dt1
# for one year training - add 252
# to evaluate for one month outside the training window - add 273
# to predict for the first day outside of the evaluation window (have to lag 21) - add 294
dt_end = (len(dates) - 273 - 21) - 1
# dt_end = 505
rpeat = len(etf_data['sym'].unique())
# etf_predict_file = os.path.join(data_path,'etf_pred_xgboost_binary_reg_20180928.csv')

k = 0

In [ ]:
for i in range(dt1,dt_end):
    print("Building model to predict for ",dates[i+273])
    x_train = etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),include_columns] #train with 12 mos
    y_train = etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),['target']] #train with 12 mos
#     x_eval = etf_data.loc[((etf_data['Date']>=dates[i+231]) & (etf_data['Date']<dates[i+252])),include_columns] #eval with 1 mos
#     y_eval = etf_data.loc[((etf_data['Date']>=dates[i+231]) & (etf_data['Date']<dates[i+252])),['rank_p_L-21']] #eval with 1 mos
#     x_train, x_eval, y_train, y_eval = train_test_split\
#     (etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),include_columns],\
#      etf_data.loc[((etf_data['Date']>=dates[i]) & (etf_data['Date']<dates[i+252])),['target']],\
#      test_size=0.2, random_state=54321)
#     x_eval = x_train
#     y_eval = y_train
#predict one day
    x_test = etf_data.loc[(etf_data['Date']==dates[i+273]),include_columns] #predict one day-must be 21 days removed from training
    y_test = etf_data.loc[(etf_data['Date']==dates[i+273]),['target']] #predict if etf >= market
    returns = etf_data.loc[(etf_data['Date']==dates[i+273]),['delta_p_L-21']]
    mkt_return = etf_data.loc[(etf_data['Date']==dates[i+273]),['ivv_delta_p_L-21']]
    x_train_nmpy = x_train.as_matrix()
    y_train_nmpy = y_train.as_matrix()
#     x_eval_nmpy = x_eval.as_matrix()
#     y_eval_nmpy = y_eval.as_matrix()
    x_test_nmpy = x_test.as_matrix()
    y_test_nmpy = y_test.as_matrix()
    returns_nmpy = returns.as_matrix()
    mkt_return_nmpy = mkt_return.as_matrix()


    lr = LogisticRegression(penalty='l2',C=0.8,random_state=54321)

    lr.fit(x_train_nmpy, np.ravel(y_train_nmpy))
    
    y_pred_model = lr.predict_proba(x_test_nmpy)[:,1]
    y_train_model = lr.predict(x_train_nmpy)
    y_test_model = lr.predict(x_test_nmpy)

    y_check = np.column_stack((y_test_nmpy, y_pred_model,mkt_return_nmpy,returns_nmpy))
    y_check_df = pd.DataFrame(y_check,columns=['true_value','pred_value','mkt_return','returns'])
    y_check_df.sort_values('pred_value',inplace=True,ascending=False)
    y_check_df.reset_index(inplace=True,drop=True)
    t_hold = y_check_df['pred_value'][14]
    
    model_coef = pd.DataFrame({'Feature':include_columns,'Coefficient': np.ravel(lr.coef_)})
    model_coef.sort_values(by='Coefficient',inplace=True,ascending=False) 
    model_coef.reset_index(drop=True,inplace=True)
    print('*****************************')
    print('Variables with largest coefficients:')
    print(model_coef.head(5))
    print(model_coef.tail(5))
    print('*****************************')
    print('roc_auc_score on training: {:.4f}'.format(roc_auc_score(y_train_nmpy,y_train_model)))
    print('roc_auc_score on testing: {:.4f}'.format(roc_auc_score(y_test_nmpy,y_test_model)))
    print('*****************************')
    print('Confusion Matrix Threshold: {:.6f}'.format(t_hold))
    print('*****************************')
    print('Model Eval Results:')
    print('*****************************')
    confusion = confusion_matrix(y_test_nmpy, (y_pred_model>=t_hold).astype(int))
#     fpr, tpr, thresholds = roc_curve(y_test_nmpy, (y_model_array>0.5).astype(int), pos_label=1)
    precision = confusion[1,1]/(confusion[1,1]+confusion[0,1])
    print('Confusion Matrix:\n',confusion)
    print('Precision:',round(precision,4))

    print("Model Top Five Picks")
    print(" Avg rtn top 5:", round(y_check_df['returns'][0:5].mean(),4))
    print(y_check_df.head(5))
    print("Model 2nd Five Picks")
    print(" Avg rtn 2nd 5:", round(y_check_df['returns'][5:10].mean(),4))
    print(y_check_df.iloc[5:10,])
    print("Model 3rd Five Picks")
    print(" Avg rtn 3rd 5:", round(y_check_df['returns'][10:15].mean(),4))
    print(y_check_df.iloc[10:15,])    
    print("Model Bottom Five Picks")
    print(" Avg rtn bottom 5:", round(y_check_df['returns'][-5:].mean(),4))
    print(y_check_df.tail(5))
    print('********************************************************')
    etf_predict_file = os.path.join(data_path,('etf_pred_logistic_regression_20181002_'+str(k%21)+'.csv'))
    if k < 21:
        predict_data = etf_data.loc[(etf_data['Date']==dates[i+273]),['Date','sym','target']]
        predict_data['predict'] = y_pred_model
        predict_data.to_csv(etf_predict_file)
    else:
        predict_data = etf_data.loc[(etf_data['Date']==dates[i+273]),['Date','sym','target']]
        predict_data['predict'] = y_pred_model
        predict_data.to_csv(etf_predict_file,header=False,mode='a')
    k+=1

Building model to predict for  2015-02-05
*****************************
Variables with largest coefficients:
   Coefficient                 Feature
0     3.725513      w63_rank_p_L42_avg
1     3.725513      w63_rank_v_L42_avg
2     3.472036  ivv_w42_rank_p_L84_avg
3     3.472036  ivv_w42_rank_v_L84_avg
4     2.521587     w42_rank_v_L252_avg
     Coefficient                  Feature
355    -3.168311  ivv_w21_rank_p_L189_avg
356    -3.365727      w63_rank_p_L252_avg
357    -3.365727      w63_rank_v_L252_avg
358    -3.388579       w84_rank_v_L84_avg
359    -3.388579       w84_rank_p_L84_avg
*****************************
roc_auc_score on training: 0.6664
roc_auc_score on testing: 0.5000
*****************************
Confusion Matrix Threshold: 0.314949
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[103   3]
 [ 35  12]]
Precision: 0.8
Model Top Five Picks
 Avg rtn top 5: 0.0483
   true_value  pred_value  mkt_return   returns
0         1.

*****************************
Variables with largest coefficients:
   Coefficient                 Feature
0     3.671031      w63_rank_p_L42_avg
1     3.671031      w63_rank_v_L42_avg
2     3.476334  ivv_w42_rank_v_L84_avg
3     3.476334  ivv_w42_rank_p_L84_avg
4     2.755423     w42_rank_p_L252_avg
     Coefficient                  Feature
355    -3.131807  ivv_w21_rank_v_L189_avg
356    -3.463131       w84_rank_v_L84_avg
357    -3.463131       w84_rank_p_L84_avg
358    -3.738733      w63_rank_v_L252_avg
359    -3.738733      w63_rank_p_L252_avg
*****************************
roc_auc_score on training: 0.6667
roc_auc_score on testing: 0.5000
*****************************
Confusion Matrix Threshold: 0.367016
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[59  3]
 [79 12]]
Precision: 0.8
Model Top Five Picks
 Avg rtn top 5: 0.0147
   true_value  pred_value  mkt_return   returns
0         1.0    0.464464   -0.005375  0.073544
1         

*****************************
Variables with largest coefficients:
   Coefficient                 Feature
0     3.639920      w63_rank_p_L42_avg
1     3.639920      w63_rank_v_L42_avg
2     3.375591  ivv_w42_rank_p_L84_avg
3     3.375591  ivv_w42_rank_v_L84_avg
4     2.991202     w42_rank_p_L252_avg
     Coefficient                  Feature
355    -3.005275  ivv_w42_rank_v_L252_avg
356    -3.313514       w84_rank_p_L84_avg
357    -3.313514       w84_rank_v_L84_avg
358    -3.993055      w63_rank_v_L252_avg
359    -3.993055      w63_rank_p_L252_avg
*****************************
roc_auc_score on training: 0.6673
roc_auc_score on testing: 0.5000
*****************************
Confusion Matrix Threshold: 0.256036
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[64  4]
 [74 11]]
Precision: 0.7333
Model Top Five Picks
 Avg rtn top 5: 0.0369
   true_value  pred_value  mkt_return   returns
0         1.0    0.407722   -0.002977  0.108767
1      

*****************************
Variables with largest coefficients:
   Coefficient                 Feature
0     3.520191      w63_rank_v_L42_avg
1     3.520191      w63_rank_p_L42_avg
2     3.441164  ivv_w42_rank_p_L84_avg
3     3.441164  ivv_w42_rank_v_L84_avg
4     3.189763     w42_rank_p_L252_avg
     Coefficient              Feature
355    -3.003115   w42_rank_p_L05_avg
356    -3.195506   w84_rank_p_L84_avg
357    -3.195506   w84_rank_v_L84_avg
358    -4.056800  w63_rank_p_L252_avg
359    -4.056800  w63_rank_v_L252_avg
*****************************
roc_auc_score on training: 0.6661
roc_auc_score on testing: 0.5000
*****************************
Confusion Matrix Threshold: 0.196899
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[ 26   2]
 [112  13]]
Precision: 0.8667
Model Top Five Picks
 Avg rtn top 5: -0.0015
   true_value  pred_value  mkt_return   returns
0         1.0    0.433331   -0.024061  0.024823
1         1.0    0.384146 

*****************************
Variables with largest coefficients:
   Coefficient                 Feature
0     3.489076  ivv_w42_rank_v_L84_avg
1     3.489076  ivv_w42_rank_p_L84_avg
2     3.414515      w63_rank_p_L42_avg
3     3.414515      w63_rank_v_L42_avg
4     3.235253     w42_rank_p_L252_avg
     Coefficient              Feature
355    -3.044510   w42_rank_p_L05_avg
356    -3.059310   w84_rank_p_L84_avg
357    -3.059310   w84_rank_v_L84_avg
358    -4.105066  w63_rank_v_L252_avg
359    -4.105066  w63_rank_p_L252_avg
*****************************
roc_auc_score on training: 0.6690
roc_auc_score on testing: 0.5085
*****************************
Confusion Matrix Threshold: 0.297445
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[ 35   1]
 [103  14]]
Precision: 0.9333
Model Top Five Picks
 Avg rtn top 5: 0.0031
   true_value  pred_value  mkt_return   returns
0         1.0    0.520078   -0.022192  0.003155
1         1.0    0.519590  

...

*****************************
Variables with largest coefficients:
   Coefficient              Feature
0     4.389801   w63_rank_v_L63_avg
1     4.389801   w63_rank_p_L63_avg
2     4.075238  w84_rank_p_L126_avg
3     4.075238  w84_rank_v_L126_avg
4     3.997354  w252_rank_p_L42_avg
     Coefficient               Feature
355    -3.541800    w63_rank_p_L42_avg
356    -3.855923  w189_rank_v_L126_avg
357    -3.855923  w189_rank_p_L126_avg
358    -4.917041    w84_rank_v_L84_avg
359    -4.917041    w84_rank_p_L84_avg
*****************************
roc_auc_score on training: 0.7089
roc_auc_score on testing: 0.6944
*****************************
Confusion Matrix Threshold: 0.806918
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[ 36   2]
 [102  13]]
Precision: 0.8667
Model Top Five Picks
 Avg rtn top 5: 0.0222
   true_value  pred_value  mkt_return   returns
0         1.0    0.911786    0.002828  0.041094
1         1.0    0.897298    0.002828  

*****************************
Variables with largest coefficients:
   Coefficient              Feature
0     4.420506   w63_rank_p_L63_avg
1     4.420506   w63_rank_v_L63_avg
2     4.056339  w84_rank_p_L126_avg
3     4.056339  w84_rank_v_L126_avg
4     4.053237  w252_rank_p_L42_avg
     Coefficient               Feature
355    -3.364611    w63_rank_v_L42_avg
356    -3.851496  w189_rank_v_L126_avg
357    -3.851496  w189_rank_p_L126_avg
358    -4.834656    w84_rank_p_L84_avg
359    -4.834656    w84_rank_v_L84_avg
*****************************
roc_auc_score on training: 0.7068
roc_auc_score on testing: 0.6059
*****************************
Confusion Matrix Threshold: 0.859659
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[ 36   1]
 [102  14]]
Precision: 0.9333
Model Top Five Picks
 Avg rtn top 5: 0.0283
   true_value  pred_value  mkt_return   returns
0         1.0    0.944168    0.005506  0.049670
1         1.0    0.942048    0.005506  

*****************************
Variables with largest coefficients:
   Coefficient              Feature
0     4.629368   w63_rank_p_L63_avg
1     4.629368   w63_rank_v_L63_avg
2     4.035549  w84_rank_p_L126_avg
3     4.035549  w84_rank_v_L126_avg
4     3.863287  w252_rank_p_L42_avg
     Coefficient               Feature
355    -3.263154    w63_rank_p_L42_avg
356    -3.794352  w189_rank_v_L126_avg
357    -3.794352  w189_rank_p_L126_avg
358    -4.719576    w84_rank_p_L84_avg
359    -4.719576    w84_rank_v_L84_avg
*****************************
roc_auc_score on training: 0.7065
roc_auc_score on testing: 0.5248
*****************************
Confusion Matrix Threshold: 0.617458
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[54  4]
 [84 11]]
Precision: 0.7333
Model Top Five Picks
 Avg rtn top 5: 0.0307
   true_value  pred_value  mkt_return   returns
0         1.0    0.788983     0.01652  0.055996
1         0.0    0.765853     0.01652  0.00

*****************************
Variables with largest coefficients:
   Coefficient              Feature
0     4.832300   w63_rank_v_L63_avg
1     4.832300   w63_rank_p_L63_avg
2     3.958238  w84_rank_p_L126_avg
3     3.958238  w84_rank_v_L126_avg
4     3.722047  w252_rank_p_L42_avg
     Coefficient               Feature
355    -3.064886    w63_rank_p_L42_avg
356    -3.744223  w189_rank_v_L126_avg
357    -3.744223  w189_rank_p_L126_avg
358    -4.742738    w84_rank_v_L84_avg
359    -4.742738    w84_rank_p_L84_avg
*****************************
roc_auc_score on training: 0.7074
roc_auc_score on testing: 0.5057
*****************************
Confusion Matrix Threshold: 0.367125
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[59  7]
 [79  8]]
Precision: 0.5333
Model Top Five Picks
 Avg rtn top 5: 0.0392
   true_value  pred_value  mkt_return   returns
0         1.0    0.672074    0.018045  0.047996
1         0.0    0.499770    0.018045  0.00

*****************************
Variables with largest coefficients:
   Coefficient              Feature
0     5.083997   w63_rank_p_L63_avg
1     5.083997   w63_rank_v_L63_avg
2     3.763886  w84_rank_v_L126_avg
3     3.763886  w84_rank_p_L126_avg
4     3.415215  w252_rank_p_L42_avg
     Coefficient               Feature
355    -3.419972   w63_rank_p_L189_avg
356    -3.608737  w189_rank_v_L126_avg
357    -3.608737  w189_rank_p_L126_avg
358    -4.805898    w84_rank_v_L84_avg
359    -4.805898    w84_rank_p_L84_avg
*****************************
roc_auc_score on training: 0.7064
roc_auc_score on testing: 0.4915
*****************************
Confusion Matrix Threshold: 0.500358
*****************************
Model Eval Results:
*****************************
Confusion Matrix:
 [[67  8]
 [71  7]]
Precision: 0.4667
Model Top Five Picks
 Avg rtn top 5: 0.028
   true_value  pred_value  mkt_return   returns
0         0.0    0.744969    0.007567 -0.003577
1         1.0    0.628130    0.007567  0.044